## 1. MNIST + CNN

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import numpy as np

print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


In [2]:
# load data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [3]:
# preprocess data
train_images = np.expand_dims(train_images, -1).astype('float32') / 255.
train_labels = to_categorical(train_labels, num_classes=10)

test_images = np.expand_dims(test_images, -1).astype('float32') / 255.
test_labels = to_categorical(test_labels, num_classes=10)

train_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [4]:
# generate model
model = keras.models.Sequential()
model.add( keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28,28,1)) )
model.add( keras.layers.MaxPool2D() )
model.add( keras.layers.Conv2D(64, 3, activation='relu') )
model.add( keras.layers.MaxPool2D() )
model.add( keras.layers.Conv2D(64, 3, activation='relu') )

model.add( keras.layers.Flatten() )
model.add( keras.layers.Dense(64, activation='relu') )
model.add( keras.layers.Dense(10, activation='softmax') )

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [5]:
# train model
model.compile(loss=keras.losses.categorical_crossentropy, metrics=[keras.metrics.categorical_accuracy])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 22s 372us/sample - loss: 0.1682 - categorical_accuracy: 0.94807s - loss: 0.3516 - categorical_accura - ETA: 16s - loss: 0.3318 - categor - ETA: 14 - ETA: 
Epoch 2/5
60000/60000 [==============================] - 20s 334us/sample - loss: 0.0467 - categorical_accuracy: 0.9850- loss: 0.0471 - categorical_
Epoch 3/5
60000/60000 [==============================] - 20s 333us/sample - loss: 0.0319 - categorical_accuracy: 0.9901
Epoch 4/5
60000/60000 [==============================] - 20s 334us/sample - loss: 0.0247 - categorical_accuracy: 0.9921
Epoch 5/5
60000/60000 [==============================] - 20s 333us/sample - loss: 0.0195 - categorical_accuracy: 0.9938


In [6]:
# evaluate model
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 2s 196us/sample - loss: 0.0232 - categorical_accuracy: 0.9924


0.9924

## 2. IMDB + MLP with Embedding

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

import numpy as np

print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


In [8]:
# load data
max_features = 10000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train.shape, y_train.shape

((25000,), (25000,))

In [9]:
# preprocess data
max_len = 20

print(f'len of 1st sequence before padding : {len(x_train[0])}')
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)
print(f'len of 1st sequence after padding  : {len(x_train[0])}')

len of 1st sequence before padding : 218
len of 1st sequence after padding  : 20


In [10]:
# generate model
model = keras.models.Sequential()
model.add( keras.layers.Embedding(input_dim=max_features, output_dim=8, input_length=max_len) )
model.add( keras.layers.Flatten() )
model.add( keras.layers.Dense(1, activation='sigmoid', ) )

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [11]:
# train model
model.compile(loss=keras.losses.binary_crossentropy, metrics=[keras.metrics.binary_accuracy])
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 122us/sample - loss: 0.6725 - binary_accuracy: 0.6116 - val_loss: 0.6253 - val_binary_accuracy: 0.6914
Epoch 2/10
20000/20000 [==============================] - 2s 102us/sample - loss: 0.5458 - binary_accuracy: 0.7494 - val_loss: 0.5269 - val_binary_accuracy: 0.7288
Epoch 3/10
20000/20000 [==============================] - 2s 103us/sample - loss: 0.4619 - binary_accuracy: 0.7866 - val_loss: 0.4998 - val_binary_accuracy: 0.7468
Epoch 4/10
20000/20000 [==============================] - 2s 103us/sample - loss: 0.4226 - binary_accuracy: 0.8062 - val_loss: 0.4956 - val_binary_accuracy: 0.7532
Epoch 5/10
20000/20000 [==============================] - 2s 102us/sample - loss: 0.3973 - binary_accuracy: 0.8214 - val_loss: 0.4955 - val_binary_accuracy: 0.7564
Epoch 6/10
20000/20000 [==============================] - 2s 102us/sample - loss: 0.3766 - binary_accuracy: 0.8330 - val_loss: 0.49